In [1]:
import sys
#Change path specificly to your directories
sys.path.insert(1, '/home/codahead/Fishial/FishialReaserch')

import time
import torch
import os
import cv2
import matplotlib.pyplot as plt
import torchvision.models as models
import numpy as np
from torch import nn
from torch.optim import Optimizer, SGD
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import transforms
from sklearn.neighbors import KDTree

from module.segmentation_package.src.utils import get_dataset_dicts
from module.classification_package.src.utils import save_json
from module.classification_package.interpreter_classifier import ClassifierFC
from module.classification_package.interpreter_embeding import EmbeddingClassifier
from module.segmentation_package.interpreter_segm import SegmentationInference
# from module.segmentation_package.src.utils import resize_image

#@title do detection and display result
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
import cv2


from PIL import Image
import numpy as np
import random
import sklearn.metrics.pairwise
import scipy.spatial.distance
import copy

import json
import time
import requests
import warnings
warnings.filterwarnings('ignore')
from os import listdir
from os.path import isfile, join
from shapely.geometry import Polygon
from tqdm import tqdm


def get_image(path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

def get_poly_from_coco(polys):
    poly_arrays = []
    for inst in polys:
        poly = inst['segmentation'][0]
        poly_array = []
        for i in range(int(len(poly)/2)):
            poly_array.append( (int(poly[2 * i]), int(poly[2 * i + 1])) )
        poly_arrays.append(Polygon(poly_array))
    return poly_arrays

def get_poly_from_custom(polys):
    poly_arrays = []
    for poly in polys:
        poly_array = []
        for i in range(int(len(poly)/2)):
            poly_array.append( ( int(poly[f"x{i + 1}"]), int(poly[f"y{i + 1}"]) ))
        poly_arrays.append([Polygon(poly_array), False])
    return poly_arrays

def get_best_iou(poly, data):
    max_iou = 0
    MIN_IOU = 0.4
    for i in range(len(data)):
        intersect = poly.intersection(data[i][0]).area
        union = poly.union(data[i][0]).area
        iou = intersect / union
        if iou > max_iou and iou > MIN_IOU:
            max_iou = iou
            data[i][1] = True
    return max_iou


In [ ]:
model_segmentation = SegmentationInference('FishialReaserch/best_scores/model_0067499_amp_on-Copy1.pth', device='cuda')

In [2]:
dataset_val = get_dataset_dicts('FishialReaserch/datasets/fishial_collection/cache', "Test", 
                  json_file="FishialReaserch/datasets/fishial_collection/export.json")

error:  FishialReaserch/datasets/fishial_collection/cache/00008114.jpg
error:  FishialReaserch/datasets/fishial_collection/cache/00008313.jpg
error:  FishialReaserch/datasets/fishial_collection/cache/00008341.jpg
error:  FishialReaserch/datasets/fishial_collection/cache/00008364.jpg
Left: 23468 skip: 0

KeyboardInterrupt: 

In [ ]:
total_res = {}
sdasd = 0

for image_id in tqdm(range(len(dataset_val[:10]))):
    
    image_path = dataset_val[image_id]['file_name']
    annotations = dataset_val[image_id]['annotations']
    img = get_image(image_path)

    s_t = time.time()
    array, masks, outputs = model_segmentation.simple_inference(img)
    time
    true_poly = get_poly_from_coco(annotations)
    discovered = get_poly_from_custom(array)

    dict_with_outcome = {
        'iou': [],
        'area': [],
        'false': 0
    }
    for i in range(len(true_poly)):
        try:
            print("size: ",true_poly[i].area / (dataset_val[image_id]['height'] * dataset_val[image_id]['width']))
            iou = get_best_iou(true_poly[i], discovered)
            dict_with_outcome['iou'].append(iou)
            dict_with_outcome['area'].append(true_poly[i].area / dataset_val[image_id]['height'] * dataset_val[image_id]['width'])
        except Exception as e:
            dict_with_outcome['iou'].append(0)
            dict_with_outcome['area'].append(true_poly[i].area/ dataset_val[image_id]['height'] * dataset_val[image_id]['width'])

#     dict_with_outcome['false'] = len(discovered) - sum([ss[1] for ss in discovered])
#     sdasd += len(discovered) - sum([ss[1] for ss in discovered])
#     total_res.update({os.path.basename(image_path):dict_with_outcome})
# save_json(total_res, 'name_test.json')

In [ ]:
len(dataset_val)